1) Collect	the	following	information	from	all	of	the	pages	(and	all	of	the	
books)	on	books.toscrape.com:	
a)	Name	of	the	book<br>
b)	Price<br>
c)	Rating<br>
Note	1:	complete	the	task	twice	using	CSS	Selector	and	Xpath	approaches.<br>
Note	2:	use	functions	and	loops	whenever	applicable.<br>
Note	3:	you	may	develop	your	code	without	functions	in	the	beginning,	and	add	
functions	only	if	you	see	it	works.

In [1]:
import re
import time
import requests
import numpy as np
import pandas as pd
from textblob import TextBlob, Word
from scrapy.http import TextResponse

In [2]:
base_url="http://books.toscrape.com/"
all_pages=[base_url+"catalogue/page-{}.html".format(i) for i in range(1,51)]


In [3]:
def scraper(url):
    """CSS approach"""
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    names=response.css("h3>a::attr(title)").extract()
    ratings=response.css("p[class^='star-rating']::attr(class)").extract()
    prices=[float(i[2:]) for i in response.css("p.price_color::text").extract()]
    return names,ratings,prices


In [4]:
all_names=[]
all_prices=[]
all_ratings=[]
for i in all_pages:
    names,ratings,prices=scraper(i)
    all_names.extend(names)
    all_prices.extend(prices)
    all_ratings.extend(ratings)
    time.sleep(1)

In [5]:
### xpath approach
def xpath_scraper(url):
    """CSS approach"""
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    names=response.xpath('//h3/a/@title').extract()
    ratings=response.xpath('//p[contains(@class,"star-rating")]/@class').extract()
    prices=[float(i[2:]) for i in response.xpath('//p[@class="price_color"]/text()').extract()]
    return names,ratings,prices


In [6]:
all_names=[]
all_prices=[]
all_ratings=[]
for i in all_pages:
    names,ratings,prices=xpath_scraper(i) ## only function name is changed.
    all_names.extend(names)
    all_prices.extend(prices)
    all_ratings.extend(ratings)
    time.sleep(1)


<b>2) Answer the following questions (show your work):</b><br>
a) What is the most popular rating?<br>
b) How many books on the website have that rating?<br>
c) What is the average price of his/her boos?<br><br>
<i>there are many ways to solve the tasks, however, we go with pandas

In [7]:
data=pd.DataFrame({"Name":all_names,"Price":all_prices,"Rating":all_ratings})

In [8]:
data.head()

,Name,Price,Rating
0,A Light in the Attic,51.77,star-rating Three
1,Tipping the Velvet,53.74,star-rating One
2,Soumission,50.10,star-rating One
3,Sharp Objects,47.82,star-rating Four
4,Sapiens: A Brief History of Humankind,54.23,star-rating Five


In [9]:
print(data["Rating"].value_counts())

print("\n Average price  \n",data[data['Rating']=='star-rating One'].mean())

star-rating One      226
star-rating Three    203
star-rating Five     196
star-rating Two      196
star-rating Four     179
Name: Rating, dtype: int64

 Average price  
 Price    34.561195
dtype: float64


a,b,c)<br>star-rating One whit 226 appearances is the most popular one and mean is 34.32

<b>
3) Scrape all the book descriptions from the website. Suggested workflow:</b><br>
a) define a function that scrapes all the individual book page URLs<br>
b) define a function which takes individual page URL as an input and
scrapes description.<br>
c) use function 1 to get all URLs and function.2 to get all descriptions from
those URLs.

In [10]:
# a)
def individual_url(url):
    """getting individual pages from one page"""
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    individual_pages=[base_url+'catalogue/'+i for i in response.css('.image_container>a::attr(href)').extract()]
    return individual_pages


In [11]:
# b
def get_description(url):
    page = requests.get(url)
    response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
    text=response.css("article[class='product_page']>p::text").extract_first()
    return text

In [12]:
individual_pages=[]##getting all indiviual pages from the website
for i in all_pages:
    individual_pages.extend(individual_url(i))
    time.sleep(2)

In [13]:
descriptions=[]##getting all descriptions
for i in individual_pages:
    description=get_description(i)
    descriptions.append(description)

In [14]:
descriptions[:2]

["It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot it in for you. Shel, you never sound

<b>4) Answer the following questions (most probably using RegEx):</b><br>
a) How many books have a numeric character in title?<br>
b) How many books have a year in title?<br>
c) How many books have an enumerated character (starting with # and
followed by a number) in title?

In [15]:
###we already have data, in your case most probably it is in a list, so I will make a column to a list before solving the task
print(data.head())
names=data.Name.tolist()

                                    Name  Price             Rating
0                   A Light in the Attic  51.77  star-rating Three
1                     Tipping the Velvet  53.74    star-rating One
2                             Soumission  50.10    star-rating One
3                          Sharp Objects  47.82   star-rating Four
4  Sapiens: A Brief History of Humankind  54.23   star-rating Five


In [16]:
#a,b,c
count_num_char=0
count_year=0
count_enum=0
for i in names:
    digits=re.findall("[\d]",i)# looking for numeric character in title?
    years=re.findall("[\d]{4}",i)#exactly 4 digits
    enumchar=re.findall("#\d",i)#exactly for digits
    if len(digits)>0:
        count_num_char+=1
    if len(years)>0:
        count_year+=1
    if len(enumchar)>0:
        count_enum+=1


In [17]:
print("Books with numeric characters-{}\n with year-{},\n with # -{}".format(count_num_char,count_year,count_enum))

Books with numeric characters-365
 with year-8,
 with # -313
